In [1]:
import pandas as pd

In [2]:
bank_df = pd.read_csv('bank.csv')

In [3]:
bank_df[0:5]

,age,job,marital,education,default,balance,housing-loan,personal-loan,current-campaign,previous-campaign,subscribed
0,30,unemployed,married,primary,no,1787,no,no,1,0,no
1,33,services,married,secondary,no,4789,yes,yes,1,4,no
2,35,management,single,tertiary,no,1350,yes,no,1,1,no
3,30,management,married,tertiary,no,1476,yes,yes,4,0,no
4,59,blue-collar,married,secondary,no,0,yes,no,1,0,no


In [4]:
bank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 11 columns):
age                  4521 non-null int64
job                  4521 non-null object
marital              4521 non-null object
education            4521 non-null object
default              4521 non-null object
balance              4521 non-null int64
housing-loan         4521 non-null object
personal-loan        4521 non-null object
current-campaign     4521 non-null int64
previous-campaign    4521 non-null int64
subscribed           4521 non-null object
dtypes: int64(4), object(7)
memory usage: 388.6+ KB


In [5]:
bank_df.subscribed.value_counts()

no     4000
yes     521
Name: subscribed, dtype: int64

In [6]:
from sklearn.utils import resample
bank_df_subscribed_no = bank_df[bank_df['subscribed']=='no']
bank_df_subscribed_yes = bank_df[bank_df['subscribed']=='yes']

In [7]:
 bank_df_minority_upsampled = resample(bank_df_subscribed_yes, replace=True, n_samples=2000)

In [8]:
bank_df_minority_upsampled.shape

(2000, 11)

In [9]:
new_bank_df = pd.concat([bank_df_subscribed_no,bank_df_minority_upsampled])

In [10]:
new_bank_df.shape

(6000, 11)

In [11]:
from sklearn.utils import shuffle
new_bank_df = shuffle(new_bank_df)

In [12]:
Y = new_bank_df.subscribed.map( lambda x: (int(x=='yes')))

In [13]:
encoded_bank_df = new_bank_df.drop('subscribed',axis=1)
X = pd.get_dummies(encoded_bank_df)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=42)

In [16]:
from sklearn.ensemble import GradientBoostingClassifier
logit = GradientBoostingClassifier()
logit.fit(X_train, Y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [17]:
Y_pred = logit.predict(X_test)

In [18]:
from sklearn.metrics import confusion_matrix, classification_report
confusion_matrix(Y_test, Y_pred)

array([[1112,   80],
       [ 360,  248]])

In [19]:
classification_report(Y_test, Y_pred)

'              precision    recall  f1-score   support\n\n           0       0.76      0.93      0.83      1192\n           1       0.76      0.41      0.53       608\n\n   micro avg       0.76      0.76      0.76      1800\n   macro avg       0.76      0.67      0.68      1800\nweighted avg       0.76      0.76      0.73      1800\n'